<FONT size="5.9pt"> **NB1 - PROSPECTIVE CONSEQUENTIAL BACKGROUND DATABASES (ECOINVENT 3.9)**
</FONT> 


This jupyter notebook was used to generate the future background databases for the consequential prospective LCA study of bark combustion versus biorefinery using the [*premise*](https://github.com/polca/premise) Python library. The use of *premise* requires a unique **"decryption key"** to run the code that can be obtained by contacting the developer (see documentation). 

In [1]:
import os
from shutil import copyfile
import pathlib
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# define the initial working directory
wdir = Path().resolve() # get the directory of the notebook
pdir = os.path.join(wdir,'projects')
print(f"Working Directory: {wdir}")

# set the BW project directory
pathlib.Path(pdir).mkdir(parents=True, exist_ok=True)
os.environ['BRIGHTWAY2_DIR'] = pdir
print('Project directory:', os.environ['BRIGHTWAY2_DIR'])

# set the directory to store the results
result_dir = os.path.join(wdir,'results')   
pathlib.Path(result_dir).mkdir(parents=True, exist_ok=True)
print('Results directory:', result_dir)

# set the directory in which the databases are stored
db_dir = os.path.join(wdir,'databases')
pathlib.Path(db_dir).mkdir(parents=True, exist_ok=True)
print('Databases directory:', db_dir)

Working Directory: /home/leabraud/Documents/LCAbark
Project directory: /home/leabraud/Documents/LCAbark/projects
Results directory: /home/leabraud/Documents/LCAbark/results
Databases directory: /home/leabraud/Documents/LCAbark/databases


In [3]:
import lca_algebraic as alg
import brightway2 as bw

Using environment variable BRIGHTWAY2_DIR for data directory:
/home/leabraud/Documents/LCAbark/projects


## Set up a project

In [4]:
# Create or select a project 
project_name = 'lca_bark'
bw.projects.set_current(project_name)

if project_name in bw.projects:
    bw.projects.set_current(project_name)
    print('Current project: %s  \nLocation: %s' % (bw.projects.current, bw.projects.dir))
else: 
    bw.projects.create_project(project_name)
    bw.projects.set_current(project_name)
    print('Current project: %s  \nLocation: %s' %  (bw.projects.current, bw.projects.dir))

# Check that the project was added to the directory
print(bw.projects)

Current project: lca_bark  
Location: /home/leabraud/Documents/LCAbark/projects/lca_bark.eb0fa584e58a983f7bce69d910b05036
Brightway2 projects manager with 4 objects:
	default
	lca_bark
	lcabark
	test_project
Use `projects.report()` to get a report on all projects.


## Import biosphere3

In [5]:
if 'biosphere3' in bw.databases:
        print('biosphere3 has already been imported in the project.')
else:
    bw.bw2setup()

biosphere3 has already been imported in the project.


## Import ecoinvent 3.9 consequential

In [6]:
from  bw2io import SingleOutputEcospold2Importer

ei_dir = str(str(db_dir)+'/ecoinvent 3.9_consequential_ecoSpold02/datasets')
print(ei_dir)

if 'ecoinvent_3.9_conseq' in bw.databases:
        print("ecoinvent 3.9 consequential has already been imported")
    
else:
    ei = SingleOutputEcospold2Importer(ei_dir, 'ecoinvent_3.9_conseq',use_mp=False)
    ei.apply_strategies()
    ei.statistics()
    ei.write_database()
    
bw.databases

/home/leabraud/Documents/LCAbark/databases/ecoinvent 3.9_consequential_ecoSpold02/datasets
ecoinvent 3.9 consequential has already been imported


Databases dictionary with 40 objects, including:
	Foreground DB
	Foreground DB IMAGE SSP2 Base
	Foreground DB REMIND
	Foreground DB REMIND SSP1 PkBudg500
	Foreground DB REMIND SSP2 Base
	Foreground DB ei39
	Foreground DB ei39 AT
	Foreground DB ei39 BE
	Foreground DB ei39 DE
	Foreground DB ei39 DK
Use `list(this object)` to get the complete list.

## Generate future versions of ecoinvent using premise

In [ ]:
from premise import *

In [ ]:
clear_cache()

In [ ]:
# the values can be changed - see https://chemrxiv.org/engage/chemrxiv/article-details/63ee10cdfcfb27a31fe227d
args = {
    "range time":0, #2
    "duration":0,
    "foresight":False,
    "lead time":False,
    "capital replacement rate":False,
    "measurement": 0,
    "weighted slope start": 0.75,
    "weighted slope end": 1.00
}

In [ ]:
ndb = NewDatabase(
    scenarios=[
        #{"model":"remind", "pathway":"SSP2-PkBudg500", "year":2045},
        #{"model":"remind", "pathway":"SSP2-Base", "year":2045},
        #{"model":"remind", "pathway":"SSP1-PkBudg500", "year":2045},
        {"model":"image", "pathway":"SSP2-Base", "year":2045}
    ],
    source_db="ecoinvent_3.9_conseq",
    source_version="3.9",
    key='', ## DECRYPTION KEY TO BE ADDED HERE
    system_model="consequential",
    system_args=args
)

In [ ]:
ndb.update("electricity")
ndb.update("fuels")
ndb.write_db_to_brightway()

In [ ]:
bw.databases